In [1]:
import pyphonetics 
from pyphonetics import Soundex
from pyphonetics import Metaphone
from pyphonetics import RefinedSoundex
import pyjarowinkler
from pyjarowinkler import distance
import spacy
import codecs
import pprint
import metaphone
from metaphone import doublemetaphone
import phonetics
import pandas as pd


def LD(s,t):
    import pprint
    if len(s)==0 :
        res = len(t)
    if len(t)==0:
        res = len(s)
    dist = [[0 for x in range(len(s)+1)] for x in range(len(t)+1)]
    for i in range(len(s)+1):
        dist[0][i] = i
    for j in range(len(t)+1):
        dist[j][0] = j
    for i in range(1,len(s)+1):
        for j in range(1,len(t)+1):
            if s[i-1]==t[j-1]:
                cost = 0
            else:
                cost = 1
            dist[j][i] = min(dist[j-1][i]+1,dist[j][i-1]+1,dist[j-1][i-1]+cost)
    res = dist[len(t)][len(s)]
    #pprint.pprint(dist)
    return res

def Similarity_DL(s,t):
    return LD(s,t) / max(len(s),len(t))

def Distance_Jaro(a,b):
    return 1-distance.get_jaro_distance(a, b, winkler=True, scaling=0.1)

def Similarity_Word_Vectors(a,b):
    nlp = spacy.load('en_core_web_lg')
    a1 = nlp.vocab[a]
    b1 = nlp.vocab[b]
    indice = a1.similarity(b1)
    return indice

def Distance_Phonetics_Representations(a,b):
    rs = RefinedSoundex()
    if len(a)==len(b):
        return rs.distance(a,b,metric="hamming")
    else:
        return rs.distance(a,b,metric="levenshtein")

def Code_Soundex(a):
    soundex = Soundex()
    return soundex.phonetics(a)

def Comparaison_Soundex(a,b):
    soundex = Soundex()
    return soundex.sounds_like(a,b)

def Similarity_Soundex(a,b):
    return Distance_Jaro(Code_Soundex(a),Code_Soundex(b))

def Code_Metaphone(a):
    metaphone = Metaphone()
    return metaphone.phonetics(a)

def Similarity_Metaphone(a,b):
    return Distance_Jaro(Code_Metaphone(a),Code_Metaphone(b))

def Double_Metaphone(a):
    return str(phonetics.dmetaphone(a))

def ConstruireHTML(liste):
    w = open("20000-30000.html", "w")
    w.write("<table border= '1'>")
    for ligne in liste:
        w.write("<tr><td>")
        w.write("</td><td>".join([str(x) for x in ligne]))
        w.write("</td><tr>")
    w.write("</table>")
    w.close()

def Trouver_Paire_Minimale(t):# t est un tuple
    for i in range(len(t[0])):
        if(t[0][i]!=t[1][i]):
            return tuple((t[0][i],t[1][i]))

def stockerEnListe(a):
    s = []
    for couple in a:
        s.append(couple)
    return s

### Nasal
def Trouver_Paire_Minimale(t):# t est un tuple
    if((('̃' not in t[0]) and ('̃' not in t[1])) or (('̃' in t[0]) and ('̃' in t[1]))):
        for i in range(len(t[0])):
            if(t[0][i]!=t[1][i]):
                tmp = min(len(t[0])-1,len(t[1])-1)
                if(i<tmp and (t[0][i+1]=='̃' or t[1][i+1]=='̃')):
                    return tuple((t[0][i]+t[0][i+1],t[1][i]+t[1][i+1]))
                else:
                    return tuple((t[0][i],t[1][i]))
    if('̃' in t[0] and '̃' not in t[1]):
        i = t[0].find('̃')
        return tuple((t[0][i-1]+t[0][i],t[1][i-1]))
    if('̃' in t[1] and '̃' not in t[0]):
        i = t[1].find('̃')
        return tuple((t[0][i-1],t[1][i-1]+t[1][i]))

In [2]:
print(Double_Metaphone("pierre"))
print(Double_Metaphone("bière"))
print(Double_Metaphone("tier"))
print(Double_Metaphone("bière"))
print(Double_Metaphone("Pierre"))
print(Double_Metaphone("frotter"))
print(Double_Metaphone("frôler"))

('PR', '')
('PR', '')
('T', 'TR')
('PR', '')
('PR', '')
('FRTR', '')
('FRLR', '')


### http://redac.univ-tlse2.fr/lexiques/glaff.html

lien pour le dictionnaire GLAFF

### Prétraitement du dictionnaire

In [3]:
Mots = []
API = []
lignes = []
traits = []
lemme = []
Frequence = []

fin = codecs.open("GLAFF/glaff-1.2.2.txt",'r',encoding="UTF-8")
for line in open("GLAFF/glaff-1.2.2.txt",'r',encoding="UTF-8"):
    line = fin.readline()
    lignes.append(line)
fin.close()

for ligne in lignes:
    l = ligne.strip().split("|")
    if((l[3]!="") and (set(l[-12:-1])!={'0'})):# La fréquence n'est pas de 0 dans tous les trois corpus FRWAC
        Mots.append(l[0])
        traits.append(l[1])
        lemme.append(l[2])
        l[3]=l[3].replace('.','')
        API.append(l[3])

In [4]:
df = pd.DataFrame({"Mots":Mots,"traits":traits,"lemme":lemme,"API":API},index = [i for i in range(1,len(Mots)+1)])
df.shape

(758733, 4)

In [5]:
df.head(10)

,Mots,traits,lemme,API
1,übercélèbre,Afpfs,übercélèbre,ybœʁselɛbʁ
2,übercélèbre,Afpms,übercélèbre,ybœʁselɛbʁ
3,aède,Ncms,aède,aɛd
4,aèdes,Ncmp,aède,aɛd
5,aïeul,Ncms,aïeul,ajœl
6,aïeuls,Ncmp,aïeul,ajœl
7,aïkidoka,Ncfs,aïkidoka,ajkidɔka
8,aïkidoka,Ncms,aïkidoka,ajkidɔka
9,aïkidokas,Ncfp,aïkidoka,ajkidɔka
10,aïkidokas,Ncmp,aïkidoka,ajkidɔka


### Problème 1: Pour les morphologies flexionnelles on ne peux pas lier directement le lemme et la transcription phonétique

http://paginaspersonales.deusto.es/abaitua/konzeptu/multext/LEX1_FR.html

###### Trouver quatre traits: verbe, nom, adj,adv

###### Trouver les formes en masculin singulier

###### Traiter le problème de sons Nasaux

In [ ]:
VM = []
VA = []
VL = []
VT = []
for i in range(len(Mots_API_lemme_traits)):    
    if(Mots_API_lemme_traits[i][1][1]=="Vmip3s-"):
        VM.append(Mots_API_lemme_traits[i][0][0])
        VA.append(Mots_API_lemme_traits[i][0][1])
        VL.append(Mots_API_lemme_traits[i][1][0])
        VT.append("Verbe en 3ième singulier")        
    if(Mots_API_lemme_traits[i][1][1]=="Vmip2s-"):
        VM.append(Mots_API_lemme_traits[i][0][0])
        VA.append(Mots_API_lemme_traits[i][0][1])
        VL.append(Mots_API_lemme_traits[i][1][0])
        VT.append("Verbe en 2ième singulier")

In [6]:
Mots_API = list(zip(Mots,API))
lemme_traits = list(zip(lemme,traits))
Mots_API_lemme_traits = list(zip(Mots_API,lemme_traits))
M = []
L = []
T = []
A = []
Information = dict()
for i in range(len(Mots_API_lemme_traits)):
    #Pour les verbes d'infinitif:
    if(Mots_API_lemme_traits[i][1][1]=="Vmn----"):
        M.append(Mots_API_lemme_traits[i][0][0])
        A.append(Mots_API_lemme_traits[i][0][1])
        L.append(Mots_API_lemme_traits[i][1][0])
        T.append("Verbe infinitif")      
    # Pour les noms communs/propres:
    if(Mots_API_lemme_traits[i][1][1]=="Ncms" or Mots_API_lemme_traits[i][1][1]=="Ncfs"):
        M.append(Mots_API_lemme_traits[i][1][0])
        A.append(Mots_API_lemme_traits[i][0][1])
        L.append(Mots_API_lemme_traits[i][1][0])
        T.append("Nom")
    # Pour les adjectifs:
    if((Mots_API_lemme_traits[i][1][1]=="Afcms-") or (Mots_API_lemme_traits[i][1][1]=="Afpms-") or (Mots_API_lemme_traits[i][1][1]=="Ai-ms-") or (Mots_API_lemme_traits[i][1][1]=="Ao-ms-") or (Mots_API_lemme_traits[i][1][1]=="As-ms-")):
        M.append(Mots_API_lemme_traits[i][1][0])
        A.append(Mots_API_lemme_traits[i][0][1])
        L.append(Mots_API_lemme_traits[i][1][0])
        T.append("Adjectif")
    # Pour les adverbes:
    if((Mots_API_lemme_traits[i][1][1]=="Rgp") or (Mots_API_lemme_traits[i][1][1]=="Rgc") or (Mots_API_lemme_traits[i][1][1]=="Rgc")or(Mots_API_lemme_traits[i][1][1]=="Rgn")):
        M.append(Mots_API_lemme_traits[i][1][0])
        A.append(Mots_API_lemme_traits[i][0][1])
        L.append(Mots_API_lemme_traits[i][1][0])
        T.append("Adverbe") 

In [7]:
df1 = pd.DataFrame({"Mots":M,"Transcription_Phonétique":A,"Lemme":L,"Genre Grammatical":T},index = [i for i in range(1,len(M)+1)])
df1.loc[10000:10010]

,Mots,Transcription_Phonétique,Lemme,Genre Grammatical
10000,crémaster,kʁemastɛʁ,crémaster,Nom
10001,crémateur,kʁematœʁ,crémateur,Nom
10002,crématiser,kʁematize,crématiser,Verbe infinitif
10003,crématiste,kʁematist,crématiste,Nom
10004,crématorium,kʁematɔʁjɔm,crématorium,Nom
10005,crambé,kʁɑ̃be,crambé,Nom
10006,crambe,kʁɑ̃b,crambe,Nom
10007,crème,kʁɛm,crème,Nom
10008,crément,kʁemɑ̃,crément,Nom
10009,crûment,kʁymɑ̃,crûment,Adverbe


In [10]:
with open("données/CorpusNet.txt",'w',encoding="UTF-8")as f:
    for i in range(len(M)):
        s = M[i]+"|"+L[i]+"|"+T[i]+"|"+A[i]
        f.write(s)
        f.write("\n")
with open("données/Corpus.txt",'w',encoding="UTF-8")as f:
    for i in range(len(M)):
        s = L[i]+"|"+T[i]+"|"+A[i]
        f.write(s)
        f.write("\n")

### Problème 2 Trouver les paires-minimales en utilisant Distance Levenshtein

In [8]:
print(len(M))
print(len(A))
print(len(T))
print(len(L))
Corpus = list(zip(M,A,T))
print(len(Corpus))

37581
37581
37581
37581
37581


In [ ]:
m = [(Corpus[i][0],Corpus[j][0]) for i in range(len(Corpus)) for j in range(i+1,len(Corpus))]
print(len(m))

In [ ]:
a = [(Corpus[i][1],Corpus[j][1]) for i in range(len(Corpus)) for j in range(i+1,len(Corpus))]
print(len(a))

In [ ]:
t = [(Corpus[i][2],Corpus[j][2]) for i in range(len(Corpus)) for j in range(i+1,len(Corpus))]
print(len(t))

In [ ]:
Corpus_Total = pd.DataFrame({"Mots":m,"Transcription Phonétique":a,"Genre Grammatical":t},index = [i for i in range(1,37582)])

In [ ]:
def isPaireMinimale()

In [9]:
def PM(Corpus):
    Paires_Minimales = []
    Paires_Minimales_API = []
    Paires_Minimales_POS = []
    La_PM = []
    for i in range(len(Corpus)):
        a = Corpus[i][0]
        b = Corpus[i][1]
        e = Corpus[i][2]
        for j in range(i+1,len(Corpus)):
            c = Corpus[j][0]
            d = Corpus[j][1]
            f = Corpus[j][2]
            if (('̃' not in b and '̃' not in d)or('̃' in b and '̃' in d)):### Pour les sons nasaux
                if((LD(b,d)==1) and (len(b)^len(d))==0):
                    Paires_Minimales.append(tuple((a,c)))
                    Paires_Minimales_API.append(tuple((b,d)))
                    Paires_Minimales_POS.append(tuple((e,f)))
                    print(tuple((b,d)))
                    La_PM.append(Trouver_Paire_Minimale(tuple((b,d))))
            else:
                if(LD(b,d)==2):
                    if('̃' in b and ((len(b)-len(d)==1))):
                        Paires_Minimales.append(tuple((a,c)))
                        Paires_Minimales_API.append(tuple((b,d)))
                        Paires_Minimales_POS.append(tuple((e,f)))
                        La_PM.append(Trouver_Paire_Minimale(tuple((b,d))))
                    if('̃' in d and ((len(d)-len(b)==1))):
                        Paires_Minimales.append(tuple((a,c)))
                        Paires_Minimales_API.append(tuple((b,d)))
                        Paires_Minimales_POS.append(tuple((e,f)))
                        La_PM.append(Trouver_Paire_Minimale(tuple((b,d))))
    return pd.DataFrame({"Mots":Paires_Minimales,"Transcription Phonétique":Paires_Minimales_API,"Genre Grammatical":Paires_Minimales_POS,"Paires Minimales":La_PM},index = [i for i in range(1,len(Paires_Minimales)+1)])                            

In [12]:
Phonétique.to_csv("données/phonétiques.csv",sep=':')
Phonétique.to_html("données/phonétiques.html")

In [16]:
l1 = Phonétique["Mots"]
print(len(l1))
l2 = Phonétique["Transcription Phonétique"]
print(len(l2))
l3 = Phonétique["Genre Grammatical"]
print(len(l3))

44928
44928
44928


In [25]:
with open("données/corpus_pm.txt",'w',encoding="utf-8")as f:
    for i in range(1,len(l1)+1):
        f.write(str(l1[i])+"|"+str(l2[i])+"|"+str(l3[i]))
        f.write('\n')

In [10]:
Phonétique = PM(Corpus)
Phonétique.shape

('aɛd', 'ald')
('aɛd', 'aʁd')
('aɛd', 'lɛd')
('aɛd', 'mɛd')
('aɛd', 'wɛd')
('aɛd', 'ʁɛd')
('aɛd', 'ʁɛd')
('aɛd', 'ʃɛd')
('ajœl', 'ajœʁ')
('jɑ̃b', 'jɑ̃g')
('ene', 'eʃe')
('ene', 'ete')
('ene', 'one')
('ene', 'ele')
('ene', 'eke')
('ai', 'aʃ')
('ai', 'aʒ')
('ai', 'aʁ')
('ai', 'aʁ')
('ai', 'aʁ')
('ai', 'as')
('ai', 'bi')
('ai', 'bi')
('ai', 'di')
('ai', 'gi')
('ai', 'ak')
('ai', 'aa')
('ai', 'ɥi')
('ai', 'ki')
('ai', 'li')
('ai', 'li')
('ai', 'mi')
('ai', 'ni')
('ai', 'wi')
('ai', 'wi')
('ai', 'fi')
('ai', 'pi')
('ai', 'pi')
('ai', 'pi')
('ai', 'ʁi')
('ai', 'ʁi')
('ai', 'si')
('ai', 'si')
('ai', 'ti')
('ai', 'vi')
('ainu', 'minu')
('asɔ̃', 'asɑ̃')
('asɔ̃', 'akɔ̃')
('asɔ̃', 'adɔ̃')
('asɔ̃', 'aʒɔ̃')
('asɔ̃', 'amɔ̃')
('asɔ̃', 'ajɔ̃')
('asɔ̃', 'usɔ̃')
('aeʁe', 'aeʁo')
('aeʁe', 'ʒeʁe')
('aeʁe', 'seʁe')
('aeʁjɔm', 'seʁjɔm')
('aeʁɔfʁɛ̃', 'aeʁɔtʁɛ̃')
('aeʁɔgʁam', 'aeʁɔgʁaf')
('aeʁɔgʁam', 'jeʁɔgʁam')
('aeʁomodelism', 'aeʁomodelist')
('aeʁo', 'akʁo')
('aeʁo', 'akʁo')
('aeʁo', 'beʁo')
('aeʁo', 'leʁo

('ako', 'aky')
('ako', 'eko')
('ako', 'eko')
('ako', 'akɛ')
('ako', 'akɛ')
('ako', 'aki')
('ako', 'aki')
('ako', 'akt')
('ako', 'ado')
('ako', 'aɲo')
('ako', 'ano')
('ako', 'apo')
('ako', 'aʁo')
('ako', 'aso')
('ako', 'aks')
('ako', 'ake')
('ako', 'aka')
('ako', 'aka')
('ako', 'alo')
('ako', 'alo')
('ako', 'amo')
('ako', 'apo')
('ako', 'akɛ')
('ako', 'ɔko')
('akɔte', 'ekɔte')
('akɔte', 'akite')
('akɔte', 'alɔte')
('akɔte', 'anɔte')
('akɔte', 'asɔte')
('akɔte', 'azɔte')
('akuʃmɑ̃', 'atuʃmɑ̃')
('akuʃe', 'akude')
('akuʃe', 'akuve')
('akuʃe', 'akuʃi')
('akude', 'akuve')
('akuʁiʁ', 'akeʁiʁ')
('akʁa', 'akʁo')
('akʁa', 'akʁo')
('akʁa', 'akʁy')
('akʁete', 'afʁete')
('akʁete', 'apʁete')
('akʁɔʃe', 'apʁɔʃe')
('akʁo', 'akʁy')
('akʁo', 'akʁy')
('akœj', 'ekœj')
('akœj', 'akœʁ')
('akyle', 'akyze')
('akyle', 'ekyle')
('akyle', 'adyle')
('akyle', 'anyle')
('aky', 'akɛ')
('aky', 'akɛ')
('aky', 'aki')
('aky', 'aki')
('aky', 'akt')
('aky', 'eky')
('aky', 'afy')
('aky', 'afy')
('aky', 'aʒy')
('aky', 'aks'

('adyʁe', 'aʃyʁe')
('advɑ̃tism', 'advɑ̃tist')
('afeaʒ', 'afnaʒ')
('afabləmɑ̃', 'afybləmɑ̃')
('afɛʁe', 'afeʁe')
('afɛʁe', 'afɔʁe')
('afɛʁe', 'afyʁe')
('afɛʁe', 'apɛʁe')
('afɛʁe', 'atɛʁe')
('afɛʁmɑ̃', 'afɛsmɑ̃')
('afɛʁmɑ̃', 'amɛʁmɑ̃')
('afɛʁmɑ̃', 'atɛʁmɑ̃')
('afɛʁism', 'afɛʁist')
('afɛʁism', 'afɔʁism')
('afɛʁist', 'afɔʁist')
('afɛtaʒ', 'afytaʒ')
('afalmɑ̃', 'afɔlmɑ̃')
('afalmɑ̃', 'analmɑ̃')
('afalmɑ̃', 'avalmɑ̃')
('afale', 'afame')
('afale', 'afane')
('afale', 'afile')
('afale', 'afɔle')
('afale', 'avale')
('afale', 'awale')
('afale', 'azale')
('afame', 'afane')
('afane', 'afəne')
('afane', 'afine')
('afane', 'efane')
('afanœʁ', 'afinœʁ')
('afeʁe', 'afɔʁe')
('afeʁe', 'afyʁe')
('afeʁe', 'aveʁe')
('afytaʒ', 'aʒytaʒ')
('afyte', 'afyʁe')
('afy', 'aʒy')
('afy', 'aft')
('afɛktif', 'efɛktif')
('afɛktivmɑ̃', 'efɛktivmɑ̃')
('afəne', 'afine')
('afəne', 'aməne')
('afəne', 'aməne')
('afɛʁme', 'afiʁme')
('afiʃaʒ', 'afilaʒ')
('afiʃaʒ', 'afinaʒ')
('afiʃaʒ', 'agiʃaʒ')
('afiʃe', 'afide')
('afiʃe', 'afide

('aʒist', 'aʁist')
('aʒwɛ̃te', 'eʒwɛ̃te')
('aʒwɛ̃te', 'apwɛ̃te')
('eʒwɛ̃te', 'epwɛ̃te')
('aʒɔ̃', 'amɔ̃')
('aʒɔ̃', 'ajɔ̃')
('aʒuʁe', 'aʒute')
('aʒuʁe', 'amuʁe')
('aʒyʁ', 'azyʁ')
('aʒutaʒ', 'aʒytaʒ')
('aʒu', 'aʒy')
('aʒu', 'atu')
('akɛn', 'alɛn')
('akɛn', 'amɛn')
('akɛn', 'avɛn')
('akɑ̃', 'alɑ̃')
('akɑ̃', 'amɑ̃')
('akɑ̃', 'amɑ̃')
('akɑ̃', 'avɑ̃')
('akɑ̃', 'avɑ̃')
('akɑ̃', 'avɑ̃')
('akɑ̃', 'ajɑ̃')
('akɑ̃', 'aʁɑ̃')
('alea', 'alfa')
('alea', 'alwa')
('alea', 'alfa')
('alea', 'alva')
('elagaʒ', 'elavaʒ')
('elegɑ̃', 'elemɑ̃')
('elegɑ̃', 'elefɑ̃')
('elage', 'elave')
('elage', 'evage')
('elage', 'blage')
('elage', 'flage')
('elage', 'flage')
('elage', 'ʃlage')
('elemɑ̃', 'elefɑ̃')
('ala', 'alb')
('ala', 'ald')
('ala', 'ale')
('ala', 'ale')
('ala', 'alø')
('ala', 'alp')
('ala', 'ama')
('ala', 'ama')
('ala', 'ana')
('ala', 'aʁa')
('ala', 'aja')
('ala', 'fla')
('ala', 'aka')
('ala', 'aka')
('ala', 'ale')
('ala', 'alo')
('ala', 'alo')
('ala', 'aʁa')
('ala', 'ɔla')
('ala', 'pla')
('ala', 'pla')
('al

('ami', 'ama')
('ami', 'ani')
('ami', 'api')
('ami', 'avi')
('ami', 'abi')
('ami', 'amo')
('amid', 'amil')
('emje', 'epje')
('emje', 'evje')
('emigʁɑ̃', 'imigʁɑ̃')
('emigʁe', 'imigʁe')
('emigʁe', 'imigʁe')
('emɛ̃se', 'epɛ̃se')
('emɛ̃se', 'evɛ̃se')
('ami', 'ama')
('ami', 'ani')
('ami', 'api')
('ami', 'avi')
('ami', 'abi')
('ami', 'amo')
('eminamɑ̃', 'iminamɑ̃')
('ama', 'ami')
('ama', 'ana')
('ama', 'aʁa')
('ama', 'aja')
('ama', 'aka')
('ama', 'aka')
('ama', 'amo')
('ama', 'aʁa')
('amɑ̃', 'amɔ̃')
('amɑ̃', 'avɑ̃')
('amɑ̃', 'avɑ̃')
('amɑ̃', 'avɑ̃')
('amɑ̃', 'ajɑ̃')
('amɑ̃', 'aʁɑ̃')
('amɑ̃', 'imɑ̃')
('amɑ̃', 'imɑ̃')
('ami', 'ani')
('ami', 'api')
('ami', 'avi')
('ami', 'abi')
('ami', 'amo')
('amɔliʁ', 'asɔliʁ')
('amɔm', 'aʁɔm')
('amɔm', 'amam')
('amɔ̃', 'ajɔ̃')
('amɔʁalmɑ̃', 'imɔʁalmɑ̃')
('amɔʁalism', 'imɔʁalism')
('amɔʁtiʁ', 'asɔʁtiʁ')
('emuʃe', 'emuʃɛ')
('emuʃe', 'emule')
('emuʃe', 'emuse')
('emuʃe', 'emuve')
('emuʃe', 'esuʃe')
('emuʃœʁ', 'emulœʁ')
('emulaʒ', 'emusaʒ')
('emule', 'emuse')
(

...

('tɑ̃dɛm', 'tɑ̃dɛʁ')
('tɑ̃ga', 'tɑ̃go')
('tɑ̃ga', 'tɑ̃ge')
('tɑ̃ʒɑ̃sjɛlmɑ̃', 'tɑ̃dɑ̃sjɛlmɑ̃')
('tɑ̃go', 'tɑ̃ge')
('tɑ̃go', 'tɑ̃to')
('tɑ̃go', 'tɑ̃to')
('tɑ̃gɔ̃', 'tɑ̃dɔ̃')
('tɑ̃g', 'tɑ̃k')
('tɑ̃g', 'tɑ̃d')
('tɑ̃g', 'jɑ̃g')
('tɑ̃ge', 'tɑ̃ke')
('tɑ̃ge', 'tɑ̃te')
('tɛnja', 'tenja')
('tanɛ̃', 'tapɛ̃')
('tanɛ̃', 'taʁɛ̃')
('tɑ̃kɛʁ', 'tɑ̃dɛʁ')
('tɑ̃k', 'tɑ̃d')
('tan', 'tao')
('tan', 'taʁ')
('tan', 'taj')
('tan', 'tal')
('tan', 'van')
('tane', 'tape')
('tane', 'take')
('tane', 'taʁe')
('tane', 'taʁe')
('tane', 'taze')
('tane', 'tɔne')
('tane', 'tyne')
('tane', 'vane')
('tanœʁ', 'tapœʁ')
('tanɛ̃', 'tapɛ̃')
('tanɛ̃', 'taʁɛ̃')
('tenɔʁize', 'tezɔʁize')
('tɑ̃ke', 'tɑ̃te')
('tɑ̃talat', 'tɑ̃talit')
('tɑ̃to', 'tɑ̃te')
('tɑ̃to', 'tɑ̃te')
('tɑ̃', 'tɛ̃')
('tɑ̃', 'tɔ̃')
('tɑ̃', 'tɛ̃')
('tɑ̃', 'tɛ̃')
('tɑ̃', 'tɔ̃')
('tɑ̃', 'vɑ̃')
('tɑ̃', 'vɑ̃')
('tao', 'taʁ')
('tao', 'taj')
('tao', 'tal')
('tao', 'tan')
('tao', 'tʁo')
('tao', 'tʁo')
('tɑ̃', 'tɛ̃')
('tɑ̃', 'tɔ̃')
('tɑ̃', 'tɛ̃')
('tɑ̃', 'tɛ̃')
('tɑ̃', 'tɔ̃')

('tiʒʁɔ̃', 'tigʁɔ̃')
('tiʒe', 'tije')
('tiʒe', 'tipe')
('tiʒe', 'tipe')
('tiʒe', 'tike')
('tiʒe', 'tiʁe')
('tiʒe', 'tiʁe')
('tiʒe', 'tize')
('tiʒe', 'tise')
('tigʁo', 'tigʁe')
('tigʁ', 'titʁ')
('tiki', 'tipi')
('tiki', 'tike')
('tiki', 'tikɛ')
('tiki', 'titi')
('tiki', 'wiki')
('tijaʒ', 'tiʁaʒ')
('tijaʒ', 'tizaʒ')
('tijaʒ', 'tisaʒ')
('tijaʒ', 'tujaʒ')
('tijaʒ', 'tipaʒ')
('tije', 'tipe')
('tije', 'tipe')
('tije', 'tike')
('tije', 'tiʁe')
('tije', 'tiʁe')
('tije', 'tize')
('tije', 'tise')
('tije', 'tuje')
('til', 'tiʁ')
('til', 'tit')
('til', 'tyl')
('til', 'tip')
('timɔnje', 'tizɔnje')
('timɔ̃', 'tizɔ̃')
('timɔ̃', 'titɔ̃')
('timɔ̃', 'tifɔ̃')
('timɔ̃', 'tipɔ̃')
('tinɛl', 'tynɛl')
('tɛ̃', 'tɔ̃')
('tɛ̃', 'vɛ̃')
('tɛ̃', 'vɛ̃')
('tjɔ̃', 'tʁɔ̃')
('tipe', 'tipi')
('tipe', 'tike')
('tipe', 'tiʁe')
('tipe', 'tiʁe')
('tipe', 'tize')
('tipe', 'tise')
('tipe', 'tɔpe')
('tipe', 'zipe')
('tipi', 'tipe')
('tipi', 'titi')
('tipi', 'tupi')
('tipe', 'tike')
('tipe', 'tiʁe')
('tipe', 'tiʁe')
('tipe', 'tiz

('tʁik', 'tʁij')
('tʁik', 'tʁip')
('tʁik', 'tʁɔk')
('tʁik', 'tʁɔk')
('tʁik', 'tʁyk')
('tʁik', 'tʁij')
('tʁik', 'tʁip')
('tʁik', 'tʁɔk')
('tʁik', 'tʁɔk')
('tʁik', 'tʁyk')
('tʁikɔtaʒ', 'tʁipɔtaʒ')
('tʁikɔte', 'tʁipɔte')
('tʁikɔ', 'tʁike')
('tʁikɔ', 'tʁike')
('tʁikɔte', 'tʁipɔte')
('tʁiɛl', 'tʁigl')
('tʁiɛl', 'tʁipl')
('tʁije', 'tʁime')
('tʁije', 'tʁine')
('tʁije', 'tʁijo')
('tʁije', 'tʁipe')
('tʁije', 'tʁipe')
('tʁije', 'tʁike')
('tʁije', 'tʁike')
('tʁije', 'tʁise')
('tʁije', 'tʁuje')
('tʁije', 'vʁije')
('tʁigl', 'tʁipl')
('tʁij', 'tʁip')
('tʁije', 'tʁime')
('tʁije', 'tʁine')
('tʁije', 'tʁijo')
('tʁije', 'tʁipe')
('tʁije', 'tʁipe')
('tʁije', 'tʁike')
('tʁije', 'tʁike')
('tʁije', 'tʁise')
('tʁije', 'tʁuje')
('tʁije', 'vʁije')
('tʁimaʁ', 'tʁimɛʁ')
('tʁimaʁ', 'tʁimœʁ')
('tʁimɛʁ', 'tʁimœʁ')
('tʁime', 'tʁine')
('tʁime', 'tʁipe')
('tʁime', 'tʁipe')
('tʁime', 'tʁike')
('tʁime', 'tʁike')
('tʁime', 'tʁise')
('tʁime', 'tʁome')
('tʁimɔtœʁ', 'tʁipɔtœʁ')
('tʁi', 'tʁo')
('tʁi', 'tʁo')
('tʁi', 'tʁu')
(

('vylkɛ̃', 'vylpɛ̃')
('wɛfœʁ', 'wufœʁ')
('watɛʁ', 'watœʁ')
('wɛb', 'zɛb')
('wist', 'zist')
('wist', 'zist')
('wikimedjɛ̃', 'wikipedjɛ̃')
('winɔ', 'zinɔ')
('gzilɛm', 'gzilɛn')
('jɔt', 'jɔd')
('jɔt', 'zɔt')
('jak', 'jam')
('jak', 'jas')
('jaka', 'jyka')
('jak', 'jam')
('jak', 'jas')
('jakuza', 'jakyza')
('jam', 'jas')
('jɛn', 'zɛn')
('jɔd', 'jud')
('jɔd', 'zɔd')
('jɔga', 'jɔgi')
('jɔgism', 'jɔgist')
('zaiʁ', 'zabʁ')
('zebi', 'zeby')
('zebi', 'zɔbi')
('zabʁ', 'zɛbʁ')
('zade', 'zape')
('zade', 'zape')
('zɛl', 'zɛb')
('zɛl', 'zɛf')
('zɛl', 'zɛn')
('zapatism', 'zapatist')
('zape', 'zipe')
('zape', 'zipe')
('zɛb', 'zɛf')
('zɛb', 'zɛn')
('zɛb', 'zɔb')
('zɛf', 'zɛn')
('zɛn', 'zin')
('zɛst', 'zist')
('zɛst', 'zist')
('zɛst', 'zist')
('zɛst', 'zist')
('zig', 'zin')
('zig', 'zip')
('zig', 'zin')
('zig', 'zip')
('zɛ̃kaʒ', 'zɛ̃kat')
('zɛ̃kaʒ', 'zɛ̃gaʒ')
('zin', 'zip')
('zɔb', 'zɔd')
('zɔb', 'zɔt')
('zɔd', 'zɔt')
('zom', 'zum')
('zona', 'zone')
('zɔɔlit', 'zɔɔfit')
('zume', 'zuje')
('zume', 'zuke')
(

(44928, 4)

In [91]:
Total_PM.loc[1000:1100]

,Mots,Transcription Phonétique,Genre Grammatical,Paires Minimales
1000,"(pèlerinage, pélerinage)","(pɛlʁinaʒ, pelʁinaʒ)","(Nom, Nom)","(ɛ, e)"
1001,"(palisser, palisson)","(palise, palisɔ̃)","(Verbe infinitif, Nom)","(e, ɔ̃)"
1002,"(palot, pélot)","(palo, pelo)","(Nom, Nom)","(a, e)"
1003,"(palpitant, palpiter)","(palpitɑ̃, palpite)","(Nom, Verbe infinitif)","(ɑ̃, e)"
1004,"(panard, pénard)","(panaʁ, penaʁ)","(Nom, Adverbe)","(a, e)"
1005,"(panda, pandit)","(pɑ̃da, pɑ̃di)","(Nom, Nom)","(a, i)"
1006,"(pandore, pandour)","(pɑ̃dɔʁ, pɑ̃duʁ)","(Nom, Nom)","(ɔ, u)"
1007,"(pangermanisme, pangermaniste)","(pɑ̃ʒɛʁmanism, pɑ̃ʒɛʁmanist)","(Nom, Nom)","(m, t)"
1008,"(pantomime, pantonyme)","(pɑ̃tɔmim, pɑ̃tɔnim)","(Nom, Nom)","(m, n)"
1009,"(pantouflage, pantouflard)","(pɑ̃tuflaʒ, pɑ̃tuflaʁ)","(Nom, Nom)","(ʒ, ʁ)"


### Des problèmes maintenant insolvables : coulée coulant couler coulant, mais après un filtrage et un tri des mots demandés, ça ira.

In [83]:
PM0_10000

,Mots,Transcription Phonétique,Genre Grammatical,Paires Minimales
1,"(aérogramme, aérographe)","(aeʁɔgʁam, aeʁɔgʁaf)","(Nom, Nom)","(m, f)"
2,"(aéromodélisme, aéromodéliste)","(aeʁomodelism, aeʁomodelist)","(Nom, Nom)","(m, t)"
3,"(aérophone, aérophore)","(aeʁɔfɔn, aeʁɔfɔʁ)","(Nom, Nom)","(n, ʁ)"
4,"(êta, aître)","(ɛta, ɛtʁ)","(Nom, Nom)","(a, ʁ)"
5,"(abat, ébat)","(aba, eba)","(Nom, Nom)","(a, e)"
6,"(abattre, ébattre)","(abatʁ, ebatʁ)","(Verbe infinitif, Verbe infinitif)","(a, e)"
7,"(aberger, éberger)","(abɛʁʒe, ebɛʁʒe)","(Verbe infinitif, Verbe infinitif)","(a, e)"
8,"(abolitionnisme, abolitionniste)","(abɔlisjɔnism, abɔlisjɔnist)","(Nom, Nom)","(m, t)"
9,"(abondamment, abondement)","(abɔ̃damɑ̃, abɔ̃dəmɑ̃)","(Adverbe, Nom)","(a, ə)"
10,"(abonder, aboner)","(abɔ̃de, abɔne)","(Verbe infinitif, Verbe infinitif)","(ɔ̃, ɔ)"


In [16]:
# 0-10000
Paires_Minimales = []
Paires_Minimales_API = []
Paires_Minimales_POS = []
for i in range(10000):
    for j in range(i+1,10000):
        if (('̃' not in Corpus[i][1] and '̃' not in Corpus[j][1])or('̃' in Corpus[i][1] and '̃' in Corpus[j][1])):### Pour les sons nasaux
            if((LD(Corpus[i][1],Corpus[j][1])==1) and (len(Corpus[i][1])==len(Corpus[j][1]))):
                Paires_Minimales.append(tuple((Corpus[i][0],Corpus[j][0])))
                Paires_Minimales_API.append(tuple((Corpus[i][1],Corpus[j][1])))
                Paires_Minimales_POS.append(tuple((Corpus[i][2],Corpus[j][2])))
        else:
            if((LD(Corpus[i][1],Corpus[j][1])==2) and (abs(len(Corpus[i][1])-len(Corpus[j][1]))==1)):
                Paires_Minimales.append(tuple((Corpus[i][0],Corpus[j][0])))
                Paires_Minimales_API.append(tuple((Corpus[i][1],Corpus[j][1])))
                Paires_Minimales_POS.append(tuple((Corpus[i][2],Corpus[j][2])))           
print(len(Paires_Minimales))
print(len(Paires_Minimales_API))
print(len(Paires_Minimales_POS))
print(Paires_Minimales[211],Paires_Minimales_API[211],Paires_Minimales_POS[211])

9392
9392
9392
('abord', 'alors') ('abɔʁ', 'alɔʁ') ('Nom', 'Adverbe')


In [22]:
df0 = pd.DataFrame({"Mots":Paires_Minimales,"Transcription":Paires_Minimales_API,"Genre":Paires_Minimales_POS},index = [i for i in range(1,9393)])
df0.loc[8000:8010]

,Mots,Transcription,Genre
8000,"(clam, clash)","(klam, klaʃ)","(Nom, Nom)"
8001,"(clamper, clancher)","(klɑ̃pe, klɑ̃ʃe)","(Verbe infinitif, Verbe infinitif)"
8002,"(clamper, clandé)","(klɑ̃pe, klɑ̃de)","(Verbe infinitif, Nom)"
8003,"(clamper, clapper)","(klɑ̃pe, klape)","(Verbe infinitif, Verbe infinitif)"
8004,"(clamper, clencher)","(klɑ̃pe, klɑ̃ʃe)","(Verbe infinitif, Verbe infinitif)"
8005,"(clamper, cliper)","(klɑ̃pe, klipe)","(Verbe infinitif, Verbe infinitif)"
8006,"(clamper, clipper)","(klɑ̃pe, klipe)","(Verbe infinitif, Verbe infinitif)"
8007,"(clamper, cloper)","(klɑ̃pe, klɔpe)","(Verbe infinitif, Verbe infinitif)"
8008,"(clampiner, clopiner)","(klɑ̃pine, klɔpine)","(Verbe infinitif, Verbe infinitif)"
8009,"(clamp, clin)","(klɑ̃, klɛ̃)","(Nom, Nom)"


In [12]:
# 10000-20000
Corpus = list(zip(list(zip(M,A)),T))
Paires_Minimales1 = []
Paires_Minimales_API1 = []
Paires_Minimales_POS1 = []
for i in range(10000,20000):
    for j in range(i+1,20000):
        if (('̃' not in Corpus[i][1] and '̃' not in Corpus[j][1])or('̃' in Corpus[i][1] and '̃' in Corpus[j][1]):
            if((LD(Corpus[i][1],Corpus[j][1])==1) and (len(Corpus[i][1])==len(Corpus[j][1]))):
                Paires_Minimales1.append(tuple((Corpus[i][0],Corpus[j][0])))
                Paires_Minimales1_API.append(tuple((Corpus[i][1],Corpus[j][1])))
                Paires_Minimales1_POS.append(tuple((Corpus[i][2],Corpus[j][2])))
        else:
            if((LD(Corpus[i][1],Corpus[j][1])==2) and (abs(len(Corpus[i][1])-len(Corpus[j][1])))==1):
                Paires_Minimales1.append(tuple((Corpus[i][0],Corpus[j][0])))
                Paires_Minimales_API1.append(tuple((Corpus[i][1],Corpus[j][1])))
                Paires_Minimales_POS1.append(tuple((Corpus[i][2],Corpus[j][2])))      
print(len(Paires_Minimales_API1))
print(len(Paires_Minimales_POS1))
print(len(Paires_Minimales1))
print(Paires_Minimales1[211],Paires_Minimales_API1[211],Paires_Minimales_POS1[211])

5233
5233
5233
('croton', 'crottin') ('kʁɔtɔ̃', 'kʁɔtɛ̃') ('Nom', 'Nom')


In [55]:
# 20000-30000
Corpus = list(zip(list(zip(M,A)),T))
Paires_Minimales2 = []
Paires_Minimales_API2 = []
Paires_Minimales_POS2 = []
for i in range(20000,30000):
    for j in range(i+1,30000):
        if (('̃' not in Corpus[i][1] and '̃' not in Corpus[j][1])or('̃' in Corpus[i][1] and '̃' in Corpus[j][1]):
            if((LD(Corpus[i][1],Corpus[j][1])==1) and (len(Corpus[i][1])==len(Corpus[j][1]))):
                Paires_Minimales2.append(tuple((Corpus[i][0],Corpus[j][0])))
                Paires_Minimales_API2.append(tuple((Corpus[i][1],Corpus[j][1])))
                Paires_Minimales_POS2.append(tuple((Corpus[i][2],Corpus[j][2])))
        else:
            if((LD(Corpus[i][1],Corpus[j][1])==2) and (abs(len(Corpus[i][1])-len(Corpus[j][1])))==1):
                Paires_Minimales2.append(tuple((Corpus[i][0],Corpus[j][0])))
                Paires_Minimales_API2.append(tuple((Corpus[i][1],Corpus[j][1])))
                Paires_Minimales_POS2.append(tuple((Corpus[i][2],Corpus[j][2])))     
print(len(Paires_Minimales_API2))
print(len(Paires_Minimales_POS2))
print(len(Paires_Minimales2))
print(Paires_Minimales2[211],Paires_Minimales_API2[211],Paires_Minimales_POS2[211])

6489
6489
6489
('kinnor', 'quinoa') ('kinɔʁ', 'kinɔa') ('Nom', 'Nom')


In [10]:
# 30000-37581
Corpus = list(zip(list(zip(M,A)),T))
Paires_Minimales3 = []
Paires_Minimales_API3 = []
Paires_Minimales_POS3 = []
for i in range(30000,37580):
    for j in range(i+1,40000):
        if (('̃' not in Corpus[i][1] and '̃' not in Corpus[j][1])or('̃' in Corpus[i][1] and '̃' in Corpus[j][1]):
            if((LD(Corpus[i][1],Corpus[j][1])==1) and (len(Corpus[i][1])==len(Corpus[j][1]))):
                Paires_Minimales3.append(tuple((Corpus[i][0],Corpus[j][0])))
                Paires_Minimales_API3.append(tuple((Corpus[i][1],Corpus[j][1])))
                Paires_Minimales_POS3.append(tuple((Corpus[i][2],Corpus[j][2])))
        else:
            if((LD(Corpus[i][1],Corpus[j][1])==2) and (abs(len(Corpus[i][1])-len(Corpus[j][1])))==1):
                Paires_Minimales3.append(tuple((Corpus[i][0],Corpus[j][0])))
                Paires_Minimales_API3.append(tuple((Corpus[i][1],Corpus[j][1])))
                Paires_Minimales_POS3.append(tuple((Corpus[i][2],Corpus[j][2])))     
print(len(Paires_Minimales_API3))
print(len(Paires_Minimales_POS3))
print(len(Paires_Minimales3))
print(Paires_Minimales3[211],Paires_Minimales_API3[211],Paires_Minimales_POS3[211])

4668
4668
4668
('regret', 'retrait') ('ʁəgʁɛ', 'ʁətʁɛ') ('Nom', 'Nom')


In [ ]:
# 40000-50000
Corpus = list(zip(list(zip(M,A)),T))
Paires_Minimales4 = []
Paires_Minimales_API4 = []
Paires_Minimales_POS4 = []
for i in range(40000,50000):
    for j in range(i+1,50000):
        if (('̃' not in Corpus[i][1] and '̃' not in Corpus[j][1])or('̃' in Corpus[i][1] and '̃' in Corpus[j][1]):
            if((LD(Corpus[i][1],Corpus[j][1])==1) and (len(Corpus[i][1])==len(Corpus[j][1]))):
                Paires_Minimales4.append(tuple((Corpus[i][0],Corpus[j][0])))
                Paires_Minimales_API4.append(tuple((Corpus[i][1],Corpus[j][1])))
                Paires_Minimales_POS4.append(tuple((Corpus[i][2],Corpus[j][2])))
        else:
            if((LD(Corpus[i][1],Corpus[j][1])==2) and (abs(len(Corpus[i][1])-len(Corpus[j][1])))==1):
                Paires_Minimales4.append(tuple((Corpus[i][0],Corpus[j][0])))
                Paires_Minimales_API4.append(tuple((Corpus[i][1],Corpus[j][1])))
                Paires_Minimales_POS4.append(tuple((Corpus[i][2],Corpus[j][2])))
print(len(Paires_Minimales_API4))
print(len(Paires_Minimales_POS4))
print(len(Paires_Minimales4))
print(Paires_Minimales4[211],Paires_Minimales_API4[211],Paires_Minimales_POS4[211])

In [ ]:
# 50000-60000
Corpus = list(zip(list(zip(M,A)),T))
Paires_Minimales5 = []
Paires_Minimales_API5 = []
Paires_Minimales_POS5 = []
for i in range(50000,60000):
    for j in range(i+1,60000):
        if (('̃' not in Corpus[i][1] and '̃' not in Corpus[j][1])or('̃' in Corpus[i][1] and '̃' in Corpus[j][1]):
            if((LD(Corpus[i][1],Corpus[j][1])==1) and (len(Corpus[i][1])==len(Corpus[j][1]))):
                Paires_Minimales5.append(tuple((Corpus[i][0],Corpus[j][0])))
                Paires_Minimales_API5.append(tuple((Corpus[i][1],Corpus[j][1])))
                Paires_Minimales_POS5.append(tuple((Corpus[i][2],Corpus[j][2])))
        else:
            if((LD(Corpus[i][1],Corpus[j][1])==2) and (abs(len(Corpus[i][1])-len(Corpus[j][1])))==1):
                Paires_Minimales5.append(tuple((Corpus[i][0],Corpus[j][0])))
                Paires_Minimales_API5.append(tuple((Corpus[i][1],Corpus[j][1])))
                Paires_Minimales_POS5.append(tuple((Corpus[i][2],Corpus[j][2])))
print(len(Paires_Minimales_API5))
print(len(Paires_Minimales_POS5))
print(len(Paires_Minimales5))
print(Paires_Minimales5[211],Paires_Minimales_API5[211],Paires_Minimales_POS5[211])

In [ ]:
# 60000-63207
Corpus = list(zip(list(zip(M,A)),T))
Paires_Minimales6 = []
Paires_Minimales_API6 = []
Paires_Minimales_POS6 = []
for i in range(60000,63206):
    for j in range(i+1,63206):
        if (('̃' not in Corpus[i][1] and '̃' not in Corpus[j][1])or('̃' in Corpus[i][1] and '̃' in Corpus[j][1]):
            if((LD(Corpus[i][1],Corpus[j][1])==1) and (len(Corpus[i][1])==len(Corpus[j][1]))):
                Paires_Minimales6.append(tuple((Corpus[i][0],Corpus[j][0])))
                Paires_Minimales_API6.append(tuple((Corpus[i][1],Corpus[j][1])))
                Paires_Minimales_POS6.append(tuple((Corpus[i][2],Corpus[j][2])))
        else:
            if((LD(Corpus[i][1],Corpus[j][1])==2) and (abs(len(Corpus[i][1])-len(Corpus[j][1])))==1):
                Paires_Minimales6.append(tuple((Corpus[i][0],Corpus[j][0])))
                Paires_Minimales_API6.append(tuple((Corpus[i][1],Corpus[j][1])))
                Paires_Minimales_POS6.append(tuple((Corpus[i][2],Corpus[j][2])))
print(len(Paires_Minimales_API6))
print(len(Paires_Minimales_POS6))
print(len(Paires_Minimales6))
print(Paires_Minimales6[211],Paires_Minimales_API6[211],Paires_Minimales_POS6[211])

In [7]:
def Trouver_Paire_Minimale(t):# t est un tuple
    if((('̃' not in t[0]) and ('̃' not in t[1])) or ('̃' in t[0]) and ('̃' in t[1]):
        for i in range(len(t[0])):
            if(t[0][i]!=t[1][i]):
                return tuple((t[0][i],t[1][i]))
    else:
        if('̃' in t[0]):
            i = t[0].find('̃')
            return tuple((t[0][i-1]+t[0][i],t[1][i]))
        if('̃' in t[1]):
            i = t[1].find('̃')
            return tuple((t[0][i],t[1][i-1]+t[1][i]))
print(Paires_Minimales_API[1311][0])
print(Trouver_Paire_Minimale(Paires_Minimales_API[1311]))

amɔliʁ
('m', 's')


### 3. Stocker les données et Construire le tableau

In [56]:
Mots10000 = stockerEnListe(Paires_Minimales)
APIs10000 = stockerEnListe(Paires_Minimales_API)
Traits10000 = stockerEnListe(Paires_Minimales_POS)
Mots1000020000 = stockerEnListe(Paires_Minimales1)
APIs1000020000 = stockerEnListe(Paires_Minimales_API1)
Traits1000020000 = stockerEnListe(Paires_Minimales_POS1)
Mots2000030000 = stockerEnListe(Paires_Minimales2)
APIs2000030000 = stockerEnListe(Paires_Minimales_API2)
Traits2000030000 = stockerEnListe(Paires_Minimales_POS2)
Mots3000037580 = stockerEnListe(Paires_Minimales3)
APIs3000037580 = stockerEnListe(Paires_Minimales_API3)
Traits3000037580 = stockerEnListe(Paires_Minimales_POS3)

In [59]:
with open("PM2000030000APIs.txt",'w',encoding = "UTF-8")as f:
    for ligne in APIs2000030000:
        f.write(ligne)
        f.write("\n")

In [17]:
Tableau = [[[]for x in range(5)]for y in range(len(Paires_Minimales)+1)]
Tableau[0] = ["Mots","Transcription Phonétique","POS","Double Métaphone","Paires Minimales"]
for i in range(len(Paires_Minimales)):
    Tableau[i+1][0]=str(Paires_Minimales[i])
    Tableau[i+1][1]=str(Paires_Minimales_API[i])
    Tableau[i+1][2]=str(Paires_Minimales_POS[i])
    Tableau[i+1][3]=Double_Metaphone(Paires_Minimales[i][0])+" "+Double_Metaphone(Paires_Minimales[i][1])
    Tableau[i+1][4]=str(Trouver_Paire_Minimale(Paires_Minimales_API[i]))
ConstruireHTML(Tableau)

In [19]:
Tableau1 = [[[]for x in range(5)]for y in range(len(Paires_Minimales1)+1)]
Tableau1[0] = ["Mots","Transcription Phonétique","POS","Double Métaphone","Paires Minimales"]
for i in range(len(Paires_Minimales1)):
    Tableau1[i+1][0]=str(Paires_Minimales1[i])
    Tableau1[i+1][1]=str(Paires_Minimales_API1[i])
    Tableau1[i+1][2]=str(Paires_Minimales_POS1[i])
    Tableau1[i+1][3]=Double_Metaphone(Paires_Minimales1[i][0])+" "+Double_Metaphone(Paires_Minimales1[i][1])
    Tableau1[i+1][4]=str(Trouver_Paire_Minimale(Paires_Minimales_API1[i]))
ConstruireHTML(Tableau1)

In [60]:
Tableau2 = [[[]for x in range(5)]for y in range(len(Paires_Minimales2)+1)]
Tableau2[0] = ["Mots","Transcription Phonétique","POS","Double Métaphone","Paires Minimales"]
for i in range(len(Paires_Minimales2)):
    Tableau2[i+1][0]=str(Paires_Minimales2[i])
    Tableau2[i+1][1]=str(Paires_Minimales_API2[i])
    Tableau2[i+1][2]=str(Paires_Minimales_POS2[i])
    Tableau2[i+1][3]=Double_Metaphone(Paires_Minimales2[i][0])+" "+Double_Metaphone(Paires_Minimales2[i][1])
    Tableau2[i+1][4]=str(Trouver_Paire_Minimale(Paires_Minimales_API2[i]))
ConstruireHTML(Tableau2)

In [21]:
Tableau3 = [[[]for x in range(5)]for y in range(len(Paires_Minimales3)+1)]
Tableau3[0] = ["Mots","Transcription Phonétique","POS","Double Métaphone","Paires Minimales"]
for i in range(len(Paires_Minimales3)):
    Tableau3[i+1][0]=str(Paires_Minimales3[i])
    Tableau3[i+1][1]=str(Paires_Minimales_API3[i])
    Tableau3[i+1][2]=str(Paires_Minimales_POS3[i])
    Tableau3[i+1][3]=Double_Metaphone(Paires_Minimales3[i][0])+" "+Double_Metaphone(Paires_Minimales3[i][1])
    Tableau3[i+1][4]=str(Trouver_Paire_Minimale(Paires_Minimales_API3[i]))
ConstruireHTML(Tableau3)

In [20]:
with open("Paires Minimales POS 10000.txt",'w',encoding="UTF-8")as f:
    for ligne in Paires_Minimales_POS:
        f.write(str(ligne))
        f.write('\n')